# Análise das interações dos estudantes em um MOOC

In [2]:
!pip install pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## Coleta e processamento dos dados

In [16]:
# leitura dos dados
df_2015_1 = spark.read.json("C:/Users/mulle/downloads/data/20150801-20151101-raw_user_activity.json")
df_2015_2 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20151101-20160201-raw_user_activity.json")
df_2016_1 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20160201-20160501-raw_user_activity.json")

In [17]:
df_2016_2 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20160501-20160801-raw_user_activity.json")
df_2016_3 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20160801-20170201-raw_user_activity.json")
df_2017_1 = spark.read.option("multiline","true").json("C:/Users/mulle/downloads/data/20170201-20170801-raw_user_activity.json")

In [18]:
df_2015_1.head()

AnalysisException: Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().